[![Ouvrir sur Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SkatAI/deeplearning/blob/master/notebooks/transfer_learning_inception_fr.ipynb)

# Transfer Learning avec InceptionV3 sur Chats vs Chiens

Ce notebook illustre le **transfer learning** en utilisant un modèle InceptionV3 pré-entraîné pour classifier des chats et des chiens.

**Concepts clés :**
- Transfer Learning : réutiliser les caractéristiques apprises sur ImageNet
- Couches gelées : conserver les poids pré-entraînés inchangés
- Tête personnalisée : ajouter un petit classifieur pour la classification binaire
- Augmentation de données : améliorer la généralisation
- Early Stopping : éviter le surapprentissage
- TensorBoard : suivre l'entraînement en temps réel

**Résultats attendus :**
- Précision d'entraînement : ~95%+
- Précision de validation : ~85%+ (bien meilleure qu'un entraînement from scratch)


Documentation : [https://keras.io/guides/transfer_learning/](https://keras.io/guides/transfer_learning/)

## 1. Setup : importer les bibliothèques et charger TensorBoard


In [ ]:
# Activer l'extension TensorBoard dans Jupyter
%load_ext tensorboard

# Importer les bibliothèques principales
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Importer les utilitaires
import os
import zipfile
import datetime
import matplotlib.pyplot as plt
import numpy as np

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {tf.config.list_physical_devices('GPU')}")

## 2. Charger et configurer InceptionV3 pré-entraîné

**Ce que nous faisons :**
1. Créer un modèle InceptionV3 avec une entrée de forme (150, 150, 3)
2. Retirer la couche de classification finale (`include_top=False`)
3. Charger les poids pré-entraînés d'ImageNet via Keras (`weights='imagenet'`)
4. Geler toutes les couches (`trainable=False`) pour qu'elles ne soient pas mises à jour pendant l'entraînement

**Pourquoi geler ?**
- Les caractéristiques pré-entraînées sont déjà excellentes pour détecter les bords, textures, formes, etc.
- On veut uniquement apprendre la décision finale de classification chats vs chiens
- Cela économise du calcul et évite le surapprentissage sur un petit jeu de données


In [ ]:
# Initialiser le modèle InceptionV3 avec les poids pré-entraînés d'ImageNet
# - input_shape=(150, 150, 3) : images RGB de 150x150
# - include_top=False : retirer la couche de classification finale (on ajoutera la nôtre)
# - weights='imagenet' : charger les poids pré-entraînés depuis Keras
pre_trained_model = InceptionV3(
    input_shape=(150, 150, 3),
    include_top=False,
    weights='imagenet'
)

# Geler toutes les couches pour qu'elles ne s'entraînent pas
# On veut uniquement entraîner la couche de classification finale qu'on va ajouter
for layer in pre_trained_model.layers:
    layer.trainable = False

print(f"Nombre de couches dans InceptionV3 : {len(pre_trained_model.layers)}")
print(f"Toutes les couches gelées : {not any(layer.trainable for layer in pre_trained_model.layers)}")

## 3. Examiner le modèle pré-entraîné

Regardons l'architecture et la forme de sortie avec laquelle nous allons travailler.


In [ ]:
# Récupérer la couche de sortie qu'on utilisera comme base pour notre classifieur
# 'mixed7' est l'un des derniers blocs convolutifs d'InceptionV3
last_layer = pre_trained_model.get_layer('mixed7')

print(f"Nom de la dernière couche : {last_layer.name}")
# Utiliser model.output_shape au lieu de layer.output_shape
print(f"Forme de sortie de la dernière couche : {pre_trained_model.output_shape}")
print(f"\nLes caractéristiques d'InceptionV3 ont la forme : {pre_trained_model.output_shape[1:]}")
print(f"(Ce sont 2048 cartes de caractéristiques de taille 4×4)")

print("\n" + "="*80)
print("Résumé du modèle InceptionV3 :")
print("="*80)
print(f"Nombre total de couches : {len(pre_trained_model.layers)}")
print(f"Forme de sortie du modèle : {pre_trained_model.output_shape}")


## 4. Construire la tête de classification personnalisée

On ajoute un petit réseau de neurones au-dessus d'InceptionV3 pour la classification binaire.

**Architecture :**
```
Sortie InceptionV3 (mixed7)
    ↓
GlobalAveragePooling2D → (768 neurones)
    ↓
Dense(1024) + ReLU ← Apprend la combinaison des caractéristiques
    ↓
Dropout(0.2) ← Régularisation (éviter le surapprentissage)
    ↓
Dense(1) + Sigmoid ← Sortie binaire (0=chat, 1=chien)
```

**Note :** On utilise `GlobalAveragePooling2D` au lieu de `Flatten` pour réduire drastiquement le nombre de paramètres et la consommation mémoire.


In [ ]:
# Récupérer la sortie de la dernière couche d'InceptionV3
last_output = last_layer.output

# Construire notre tête de classification personnalisée
# Étape 1 : GlobalAveragePooling2D réduit chaque carte de caractéristiques à une seule valeur
# Beaucoup plus efficace en mémoire que Flatten
x = layers.GlobalAveragePooling2D()(last_output)

# Étape 2 : Ajouter une couche dense avec 1024 neurones et activation ReLU
# Cette couche apprend à combiner les caractéristiques pour la classification
x = layers.Dense(1024, activation='relu')(x)

# Étape 3 : Ajouter du dropout pour la régularisation
# Désactive aléatoirement 20% des neurones pendant l'entraînement pour éviter le surapprentissage
x = layers.Dropout(0.2)(x)

# Étape 4 : Couche de sortie avec activation sigmoïde
# La sigmoïde produit une valeur entre 0 et 1
# 0 = chat, 1 = chien (classification binaire)
x = layers.Dense(1, activation='sigmoid')(x)

# Créer le modèle final en connectant l'entrée d'InceptionV3 à notre sortie
model = Model(pre_trained_model.input, x)

print("Modèle personnalisé créé avec succès !")
print(f"\nLe modèle final a {len(model.layers)} couches au total")
print(f"Seules les dernières couches s'entraînent (GlobalAveragePooling2D, Dense, Dropout, Dense)")

## 5. Compiler le modèle

**Paramètres de compilation :**
- **Optimiseur** : Adam avec un faible taux d'apprentissage (0.0001) car on fait du fine-tuning
- **Loss** : Binary crossentropy (pour la classification binaire)
- **Métriques** : Accuracy pour suivre les performances


In [ ]:
# Compiler le modèle
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Faible taux d'apprentissage pour le fine-tuning
    loss='binary_crossentropy',             # Loss pour la classification binaire
    metrics=['accuracy']                    # Suivre la précision pendant l'entraînement
)

print("Modèle compilé avec succès !")

# Afficher le résumé des couches entraînables
print("\nRésumé du modèle (couches principales) :")
print("="*80)
model.summary()

## 6. Télécharger et extraire le jeu de données

On télécharge le jeu de données Microsoft chats et chiens (~65 Mo compressé).


In [ ]:
# Télécharger le jeu de données
print("Téléchargement du jeu de données chats et chiens (cela peut prendre 2-5 minutes)...")
!wget https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip

# Extraire le jeu de données
print("\nExtraction du jeu de données...")
zip_ref = zipfile.ZipFile("./cats_and_dogs_filtered.zip", 'r')
zip_ref.extractall("tmp/")
zip_ref.close()

print("Jeu de données extrait avec succès !")

## 7. Configurer les chemins des données

Organiser les répertoires d'entraînement et de validation.


In [ ]:
# Définir le répertoire de base
base_dir = 'tmp/cats_and_dogs_filtered'

# Répertoires d'entraînement
train_dir = os.path.join(base_dir, 'train')
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Répertoires de validation
validation_dir = os.path.join(base_dir, 'validation')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# Compter les images
num_train_cats = len(os.listdir(train_cats_dir))
num_train_dogs = len(os.listdir(train_dogs_dir))
num_val_cats = len(os.listdir(validation_cats_dir))
num_val_dogs = len(os.listdir(validation_dogs_dir))

print("Structure du jeu de données :")
print("="*60)
print(f"Jeu d'entraînement :")
print(f"  - Chats : {num_train_cats}")
print(f"  - Chiens : {num_train_dogs}")
print(f"  - Total : {num_train_cats + num_train_dogs}")
print(f"\nJeu de validation :")
print(f"  - Chats : {num_val_cats}")
print(f"  - Chiens : {num_val_dogs}")
print(f"  - Total : {num_val_cats + num_val_dogs}")

## 8. Créer les générateurs de données avec augmentation

L'**augmentation de données** applique des transformations aléatoires aux images d'entraînement pour :
- Augmenter la taille effective du jeu de données
- Améliorer la généralisation du modèle
- Éviter le surapprentissage

**Augmentations appliquées :**
- Rotation : ±40°
- Décalage largeur/hauteur : 20%
- Zoom : 20%
- Retournement horizontal : aléatoire

**Note :** On n'augmente pas les données de validation (uniquement la normalisation) pour obtenir des métriques de performance fiables.


In [ ]:
# Générateur de données d'entraînement AVEC augmentation
# Ces transformations aident le modèle à mieux généraliser
train_datagen = ImageDataGenerator(
    rescale=1.0/255.,           # Normaliser les valeurs des pixels dans [0, 1]
    rotation_range=40,          # Rotations aléatoires jusqu'à 40 degrés
    width_shift_range=0.2,      # Décalages horizontaux aléatoires jusqu'à 20%
    height_shift_range=0.2,     # Décalages verticaux aléatoires jusqu'à 20%
    shear_range=0.2,            # Transformations de cisaillement aléatoires
    zoom_range=0.2,             # Zoom aléatoire jusqu'à 20%
    horizontal_flip=True,       # Retournements horizontaux aléatoires
    fill_mode='nearest'         # Mode de remplissage pour les pixels hors limites
)

# Générateur de données de validation SANS augmentation
# On normalise uniquement, pas d'autres transformations
# Cela garantit une évaluation sur des données réalistes non augmentées
test_datagen = ImageDataGenerator(
    rescale=1.0/255.            # Normaliser les valeurs des pixels dans [0, 1]
)

# Créer le générateur de données d'entraînement
# Charge automatiquement les images depuis les répertoires et applique l'augmentation
train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size=20,              # Charger 20 images à la fois
    class_mode='binary',        # Classification binaire (chats=0, chiens=1)
    target_size=(150, 150)      # Redimensionner toutes les images en 150×150
)

# Créer le générateur de données de validation
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    batch_size=20,              # Charger 20 images à la fois
    class_mode='binary',        # Classification binaire
    target_size=(150, 150)      # Redimensionner toutes les images en 150×150
)

print("Générateurs de données créés avec succès !")

## 9. Configurer les callbacks : Early Stopping et TensorBoard

**Early Stopping :**
- Surveille la loss de validation
- Arrête l'entraînement si la loss de validation ne s'améliore pas pendant 3 epochs
- Sauvegarde les meilleurs poids du modèle
- Évite le surapprentissage et fait gagner du temps

**TensorBoard :**
- Enregistre les métriques d'entraînement (loss, accuracy)
- Enregistre les histogrammes des poids
- Permet le suivi en temps réel et la comparaison


In [ ]:
# Créer un répertoire de logs unique pour cet entraînement
log_dir = "logs/inception_v3_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Configurer le callback Early Stopping
# Arrête l'entraînement si la loss de validation cesse de s'améliorer
early_stop = EarlyStopping(
    monitor='val_loss',         # Surveiller la loss de validation
    patience=3,                 # Arrêter si pas d'amélioration pendant 3 epochs
    restore_best_weights=True,  # Restaurer les poids de la meilleure epoch
    verbose=1                   # Afficher un message à l'arrêt
)

# Configurer le callback TensorBoard
# Enregistre les métriques d'entraînement pour la visualisation
tensorboard_callback = TensorBoard(
    log_dir=log_dir,            # Répertoire de sauvegarde des logs
    histogram_freq=1            # Sauvegarder les histogrammes des poids à chaque epoch
)

print(f"Callbacks configurés :")
print(f"  - Early Stopping : patience=3 epochs")
print(f"  - TensorBoard : {log_dir}")

## 10. Entraîner le modèle

On lance maintenant l'entraînement du modèle avec les callbacks.

**Détails de l'entraînement :**
- Seule la tête personnalisée s'entraîne (les couches d'InceptionV3 sont gelées)
- steps_per_epoch=100 : traiter 100 batchs de 20 images par epoch = ~2000 images
- validation_steps=50 : traiter 50 batchs de 20 images pour la validation = ~1000 images
- epochs=10 : maximum 10 epochs (peut s'arrêter plus tôt si l'Early Stopping se déclenche)

**Temps estimé :** 2-5 minutes par epoch sur GPU, 10-20 minutes sur CPU


In [ ]:
# Entraîner le modèle
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    steps_per_epoch=100,        # Nombre de batchs par epoch
    validation_steps=50,        # Nombre de batchs pour la validation
    epochs=10,                  # Nombre maximum d'epochs
    callbacks=[early_stop, tensorboard_callback],  # Appliquer les callbacks
    verbose=1                   # Afficher la progression
)

print("\nEntraînement terminé !")

## 11. Lancer TensorBoard

Visualiser les métriques d'entraînement, les distributions des poids et l'architecture du modèle dans TensorBoard.


In [ ]:
# Lancer TensorBoard
%tensorboard --logdir logs

print("TensorBoard est maintenant actif !")
print(f"Voir sur : http://localhost:6006")
print(f"\nVous pouvez voir :")
print(f"  - Scalars : graphiques de loss et accuracy")
print(f"  - Histograms : distributions des poids par couche")
print(f"  - Graph : architecture du modèle")

## 12. Évaluer et visualiser les résultats

Tracer les métriques d'entraînement vs validation pour évaluer les performances du modèle et le surapprentissage.


In [ ]:
# Extraire les métriques de l'historique
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Créer les numéros d'epochs
epochs = range(1, len(acc) + 1)

# Créer une figure avec 2 sous-graphiques
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Graphique 1 : Accuracy
ax1.plot(epochs, acc, 'r-', label='Accuracy entraînement', linewidth=2)
ax1.plot(epochs, val_acc, 'b-', label='Accuracy validation', linewidth=2)
ax1.set_title('Accuracy entraînement vs validation', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0, 1])

# Graphique 2 : Loss
ax2.plot(epochs, loss, 'r-', label='Loss entraînement', linewidth=2)
ax2.plot(epochs, val_loss, 'b-', label='Loss validation', linewidth=2)
ax2.set_title('Loss entraînement vs validation', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Afficher le résumé des statistiques
print("\n" + "="*80)
print("RÉSUMÉ DES RÉSULTATS D'ENTRAÎNEMENT")
print("="*80)
print(f"\nEpoch finale : {len(epochs)}")
print(f"\nMétriques d'entraînement :")
print(f"  - Accuracy : {acc[-1]:.4f} ({acc[-1]*100:.2f}%)")
print(f"  - Loss : {loss[-1]:.4f}")
print(f"\nMétriques de validation :")
print(f"  - Accuracy : {val_acc[-1]:.4f} ({val_acc[-1]*100:.2f}%)")
print(f"  - Loss : {val_loss[-1]:.4f}")
print(f"\nÉcart (indicateur de surapprentissage) :")
print(f"  - Écart d'accuracy : {(acc[-1] - val_acc[-1])*100:.2f}%")
print(f"  - Écart de loss : {loss[-1] - val_loss[-1]:.4f}")

if acc[-1] - val_acc[-1] < 0.10:
    print(f"\n✓ Bonne généralisation ! L'écart est inférieur à 10%")
elif acc[-1] - val_acc[-1] < 0.20:
    print(f"\n⚠ Surapprentissage modéré détecté. Envisagez plus de régularisation.")
else:
    print(f"\n✗ Surapprentissage significatif. Le modèle a mémorisé les données d'entraînement.")

## 13. Sauvegarder le modèle entraîné

Sauvegarder le modèle pour une utilisation future ou un déploiement.


In [ ]:
# Sauvegarder le modèle
model_save_path = 'inception_v3_cats_dogs.h5'
model.save(model_save_path)

print(f"Modèle sauvegardé dans : {model_save_path}")
print(f"\nVous pouvez le charger plus tard avec :")
print(f"  from tensorflow.keras.models import load_model")
print(f"  model = load_model('{model_save_path}')")

## 14. Tester les prédictions (optionnel)

Faire des prédictions sur quelques images de validation pour voir comment le modèle se comporte.


In [ ]:
import random
from tensorflow.keras.preprocessing import image as keras_image

# Récupérer un batch d'exemple du générateur de validation
val_images, val_labels = next(validation_generator)

# Faire les prédictions
predictions = model.predict(val_images[:8])

# Visualiser les prédictions
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, (image, label, pred) in enumerate(zip(val_images[:8], val_labels[:8], predictions[:8])):
    # Dénormaliser l'image pour l'affichage
    image_display = (image * 255).astype(np.uint8)

    # Obtenir les labels vrais et prédits
    true_label = "Chien" if label == 1 else "Chat"
    pred_prob = pred[0]
    pred_label = "Chien" if pred_prob > 0.5 else "Chat"
    confidence = max(pred_prob, 1 - pred_prob) * 100

    # Couleur : vert si correct, rouge si faux
    is_correct = (label == 1 and pred_prob > 0.5) or (label == 0 and pred_prob <= 0.5)
    color = 'green' if is_correct else 'red'

    # Afficher
    axes[i].imshow(image_display.astype('uint8'))
    axes[i].set_title(f'Vrai : {true_label}\nPréd : {pred_label} ({confidence:.1f}%)',
                     color=color, fontweight='bold')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

print("\nPrédictions d'exemple terminées !")

## Résumé

**Ce que nous avons accompli :**

1. ✓ Chargé le modèle InceptionV3 pré-entraîné (entraîné sur ImageNet)
2. ✓ Gelé toutes les couches pour préserver les caractéristiques apprises
3. ✓ Ajouté une tête de classification personnalisée (Flatten + Dense + Dropout + Dense)
4. ✓ Appliqué l'augmentation de données pour éviter le surapprentissage
5. ✓ Entraîné avec le callback Early Stopping (évite l'entraînement inutile)
6. ✓ Suivi l'entraînement avec TensorBoard
7. ✓ Obtenu une haute précision sur la classification chats vs chiens

**Avantages du Transfer Learning :**
- Entraînement beaucoup plus rapide (minutes au lieu d'heures)
- Meilleure précision avec peu de données (haute accuracy en validation)
- Exploite les connaissances de plus d'1M d'images ImageNet
- Risque réduit de surapprentissage

**Prochaines étapes :**
- Fine-tuner avec `model.trainable = True` et un très faible taux d'apprentissage pour améliorer la précision
- Tester sur vos propres images
- Déployer le modèle en production
- Expérimenter avec d'autres modèles pré-entraînés (ResNet, MobileNet, etc.)
